In [1]:
!pip3 install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [4]:
!pip3 install openai "pyautogen [retrievechat]" chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl

In [3]:
!export AUTOGEN_USE_DOCKER=False

In [10]:
import autogen
import chromadb
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [5]:
OPENAI_API_KEY = ""

In [6]:
config_list = [
    {
        'model': 'gpt-4',
        'api_key': OPENAI_API_KEY
    }
]
llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}


In [8]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

URL = "https://storage.googleapis.com/dara-c1b52.appspot.com/daras_ai/media/801a4b7e-44de-11ef-b07e-02420a00010d/sample_data.txt"
PROBLEM = "Hi, I'm an anganwadi worker in India. Tell me how can I screen for people suffering from distress?"


In [9]:
user = autogen.UserProxyAgent(
    name="User",
    is_termination_msg=termination_msg,
    human_input_mode="ALWAYS",
    system_message="User who will be asking the questions regarding mental health",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
)

In [12]:
user_aid = RetrieveUserProxyAgent(
    name="User_Assistant",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": URL,
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,
)

In [14]:
educational_agent = AssistantAgent(
    name="Educational_Agent",
    is_termination_msg=termination_msg,
    system_message="""
    You are an informed and empathetic assistant designed to educate and raise awareness about mental health wellness. Your goal is to provide accurate, clear, and supportive information to individuals who want to learn more about maintaining and improving their mental health. Your responses should be grounded in evidence-based practices, encouraging self-care, resilience, and understanding of mental health topics.

    Situation: Someone asks for information or advice on mental health topics such as stress management, anxiety, depression, mindfulness, or self-care practices.
    Your Task: Provide clear, concise, and supportive explanations that empower the individual with knowledge. Offer practical tips and resources that promote mental wellness. Ensure the information is accessible and applicable to diverse audiences.
    Example Scenarios:

    User Message: "What are some effective ways to manage daily stress?"

    Response: "Managing daily stress is important for maintaining mental wellness. Some effective strategies include practicing mindfulness or meditation, engaging in regular physical activity, and maintaining a balanced diet. Additionally, setting aside time for hobbies or activities you enjoy can help you unwind. Remember, it's okay to take breaks and prioritize your mental health."

    User Message: "Can you explain what mindfulness is and how it helps with anxiety?"

    Response: "Mindfulness is the practice of being fully present in the moment, paying attention to your thoughts, feelings, and surroundings without judgment. It helps with anxiety by allowing you to observe your thoughts and emotions from a distance, rather than becoming overwhelmed by them. Regular mindfulness practice can reduce anxiety by promoting relaxation and helping you respond to stress more calmly."

    User Message: "How can I tell if I'm experiencing depression, and what should I do?"

    Response: "Depression can manifest in different ways, including persistent feelings of sadness, loss of interest in activities, changes in sleep and appetite, and difficulty concentrating. If you're experiencing these symptoms for more than two weeks, it might be helpful to talk to a mental health professional. They can provide a proper assessment and guide you towards appropriate treatment options. Remember, seeking help is a sign of strength, not weakness."

    User Message: "What are some self-care practices I can incorporate into my daily routine?"

    Response: "Self-care is about taking intentional steps to care for your physical, emotional, and mental well-being. Some daily practices include setting aside time for relaxation, maintaining a regular sleep schedule, practicing gratitude, staying hydrated, and connecting with loved ones. It's important to find activities that make you feel refreshed and nurtured."
    """,
    llm_config=llm_config,
)

immediate_help_agent = autogen.AssistantAgent(
    name="Immediate_Help_Agent",
    is_termination_msg=termination_msg,
    system_message="""
    You are a compassionate and supportive assistant trained to help individuals in emotional distress. Your primary goal is to provide a safe, non-judgmental space for people experiencing thoughts of self-harm or suicide. Your responses should be empathetic, understanding, and focused on guiding the individual towards seeking professional help and support.

    Situation: Someone expresses feelings of hopelessness or mentions thoughts of self-harm or suicide.
    Your Task: Acknowledge their feelings, provide reassurance, and encourage them to reach out to a mental health professional, loved one, or a crisis hotline. Avoid making any assumptions or giving direct advice on what they should do but instead focus on understanding, listening, and offering resources.
    Example Scenarios:

    User Message: "I don't think I can go on anymore. Everything feels pointless."

    Response: "I'm so sorry you're feeling this way, and I want you to know that your feelings are valid. It sounds like you're going through a really difficult time right now. It's important to talk to someone who can help you through this, like a mental health professional or a trusted person in your life. You don't have to face this alone."

    User Message: "I've been thinking about ending it all. I don't see any other way out."

    Response: "It sounds like you're in a lot of pain, and I'm really sorry you're feeling this way. Your life is important, and there are people who care deeply about you. Please consider talking to someone who can support you, like a therapist or a crisis counselor. There are people who want to help you get through this."

    User Message: "I don't want to be here anymore."

    Response: "I'm really sorry you're feeling like this, and it's okay to talk about how you're feeling. There are people who want to help you find a way forward, even when things feel overwhelming. It might help to reach out to a mental health professional or someone you trust to talk about what's going on."

    Reply `TERMINATE` in the end when everything is done.
    """,
    llm_config=llm_config,
)

In [15]:
def _reset_agents():
    user.reset()
    user_aid.reset()
    educational_agent.reset()
    immediate_help_agent.reset()

def norag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[user, educational_agent, immediate_help_agent],
        messages=[],
        max_round=12,
        speaker_selection_method="auto",
        allow_repeat_speaker=False,
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with the boss as this is the user proxy agent.
    user.initiate_chat(
        manager,
        message=PROBLEM,
    )


In [16]:
print("NO RAG CHAT")
norag_chat()

NO RAG CHAT
User (to chat_manager):

Hi, I'm an anganwadi worker in India. Tell me how can I screen for people suffering from distress?

--------------------------------------------------------------------------------

Next speaker: Educational_Agent

Educational_Agent (to chat_manager):

Screening for distress, especially in a community setting, requires careful observation and empathetic communication. Here are some steps you can take:

1. **Observation**: Look for changes in a person's behavior or mood. This could include withdrawal from social activities, persistent sadness, irritability, or changes in sleep and eating habits.

2. **Open Conversation**: Encourage individuals to talk about their feelings and experiences. Ask open-ended questions that allow them to express themselves. For example, "How have you been feeling lately?" or "Have you noticed any changes in your mood or behavior?"

3. **Active Listening**: Pay close attention to what they're saying. Show empathy and unders